In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pickle
import math

In [2]:
%%HTML
<style> code {background-color : orange !important;} </style>
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
#ImageRecord = namedtuple('Img', 'val left right')
#def Node(val, left=None, right=None):
#  return NodeT(val, left, right)
#        defaultargs = (None, "img_" + str(recIndex), {})
#        img, imgName, kwargs = tuple(map(lambda x, y: y if y is not None else x, defaultargs, imgRecord))

    
#-------------------------------------------------------------------
def PlotImageRecords(imgRecords):
    fig = plt.gcf()
    fig.set_size_inches(18,12)
    #fig.set_dpi(180)

    numImages = len(imgRecords)
    numCols = 3
    numRows = math.ceil(numImages/numCols)
    for recIndex, imgRecord in enumerate(imgRecords):
        numFields = len(imgRecord)
        img = imgRecord[0]
        if (numFields >= 2):
            imgName =  imgRecord[1]
        else:
            imgName =  "img_" + str(recIndex)
            
        if (numFields >= 3):
            kwArgs =  imgRecord[2]
        else:
            kwArgs =  {}
                
        plt.subplot(numRows, numCols, recIndex+1)
        plt.title(imgName)
        #plt.axis('off')
        plt.imshow(img, **kwArgs)
        
    fig.tight_layout()
        
def Plot2ImgRecs(imgRec0, imgRec1):
    img0, imgName0 = imgRec0
    img1, imgName1 = imgRec1
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img0)
    ax1.set_title(imgName0, fontsize=50)
    ax2.imshow(img1, cmap='gray')
    ax2.set_title(imgName1, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [4]:
#-------------------------------------------------------------------
def SobelThresholdAbs(imgIn, orient='x', threshMinMax=(0, 255)):
    """
    Define a function that applies Sobel x or y, 
    then takes an absolute value and applies a threshold.
    :param img: input image as np.array
    """
    
    # 1) Convert to grayscale
    imgGray = cv2.cvtColor(imgIn,cv2.COLOR_RGB2GRAY)

    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if (orient == 'x'):
        sobelx = cv2.Sobel(imgGray, cv2.CV_64F, 1, 0)
        sobelorient = sobelx
    else:
        sobely = cv2.Sobel(imgGray, cv2.CV_64F, 0, 1)
        sobelorient = sobely
       
    
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobelorient)

    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient magnitude is > thresh_min and < thresh_max
    imgBWMask = np.zeros_like(scaled_sobel)
    imgBWMask[(scaled_sobel >= threshMinMax[0]) & (scaled_sobel <= threshMinMax[1])] = 1

    return imgBWMask

In [5]:
#-------------------------------------------------------------------
def SobelThesholdMag(imgIn, sobel_kernel=3, threshMinMax=(0, 255)):
    """
    Calculates the Sobel XY magnitude value and applies a threshold.
    :param img: input image as np.array
    """    
    # 1) Convert to grayscale
    imgGray = cv2.cvtColor(imgIn,cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(imgGray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(imgGray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*gradmag/np.max(gradmag))
    #scale_factor = np.max(gradmag)/255 
    #scaled_sobel = (gradmag/scale_factor).astype(np.uint8) 
    
    # 5) Create a binary mask where mag thresholds are met
    imgBWMask = np.zeros_like(scaled_sobel)
    imgBWMask[(scaled_sobel >= threshMinMax[0]) & (scaled_sobel <= threshMinMax[1])] = 1
    return imgBWMask

In [15]:
#-------------------------------------------------------------------
def SobelThresholdDir(imgIn, sobel_kernel=3, threshMinMax=(0, np.pi/2)):
    """
    Calculates the Sobel gradient direction thresholds
    :param img: input image as np.array
    """
    
    # 1) Convert to grayscale
    imgGray = cv2.cvtColor(imgIn,cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(imgGray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(imgGray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    sobelxabs = np.absolute(sobelx)
    sobelyabs = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    imgSobelThrDir = np.arctan2(sobelyabs, sobelxabs)
    # 5) Create a binary mask where direction thresholds are met
    min = threshMinMax[0]
    max = threshMinMax[1]
    imgBWMask = np.zeros_like(imgSobelThrDir)
    imgBWMask[(imgSobelThrDir >= min) & (imgSobelThrDir <= max)] = 1
    return imgBWMask

In [17]:
def HSLThreshold(imgHLSIn, channelNum, threshMinMax=(0, 255)):
    imgOneCh = imgHLSIn[:,:,channelNum]
    imgBWMask = np.zeros_like(imgOneCh)
    imgBWMask[(imgOneCh > threshMinMax[0]) & (imgOneCh <= threshMinMax[1])] = 255
    return imgOneCh, imgBWMask

In [20]:
imgInFileName = 'test_images/signs_vehicles_xygrad.png'
imgInFileName = 'test_images/yellowlineleft.jpg'
imgInFileName = "ImagesIn/VideosIn/project_video_frames/project_video_f0614.jpg"
imgIn = mpimg.imread(imgInFileName)
  
g_sobelKernelSize=5

imgSobelThrGradX = SobelThresholdAbs(imgIn, orient='x', threshMinMax=(30, 100))
imgSobelThrGradY = SobelThresholdAbs(imgIn, orient='y', threshMinMax=(30, 100))
imgSobelThrMag = SobelThesholdMag(imgIn, sobel_kernel=g_sobelKernelSize, threshMinMax=(30, 100))
imgSobelThrDir = SobelThresholdDir(imgIn, sobel_kernel=g_sobelKernelSize, threshMinMax=(0.8, 1.3))

#hsl(240, 5%, 76.5%)
imgHSL =  cv2.cvtColor(imgIn, cv2.COLOR_RGB2HLS)
imgH, imgHthr = HSLThreshold(imgHSL, 0, (140, 255))
imgL, imgLthr = HSLThreshold(imgHSL, 1, (90, 100))
imgS, imgSthr = HSLThreshold(imgHSL, 2, (90, 255))

imgComboSobel = np.zeros_like(imgSobelThrDir)
imgComboSobel[((imgSobelThrGradX == 1) & (imgSobelThrGradY == 1)) | ((imgSobelThrMag == 1) & (imgSobelThrDir == 1))] = 1

imgComboSatSobelX_Stack = np.dstack(( np.zeros_like(imgSobelThrGradX), imgSobelThrGradX, imgSthr)) * 255

imgComboSatSobelX_Or = np.zeros_like(imgSobelThrGradX)
imgComboSatSobelX_Or[(imgSobelThrGradX==1) | (imgSthr==1)] = 1

pipelineImages = [
    (imgH, "imgH", {"cmap":"gray"}),
    (imgL, "imgL", {"cmap":"gray"}),
    (imgS, "imgS", {"cmap":"gray"}),      
    
    (imgHthr, "imgHthr", {"cmap":"gray"}),
    (imgLthr, "imgLthr", {"cmap":"gray"}),
    (imgSthr, "imgSthr", {"cmap":"gray"}),      

    (imgIn, imgInFileName),
    (imgHSL, "imgHSL"),
    
    (imgSobelThrGradX, "imgSobelThrGradX", {"cmap":"gray"}),
    (imgSobelThrGradY, "imgSobelThrGradY", {"cmap":"gray"}),
    (imgSobelThrMag, "imgSobelThrMag", {"cmap":"gray"}),
    (imgSobelThrDir, "imgSobelThrDir", {"cmap":"gray"}),
    
    (imgComboSobel, "imgComboSobel", {"cmap":"gray"}),
    (imgComboSatSobelX_Stack, "imgComboSatSobelX_Stack", {"cmap":"gray"}),
    (imgComboSatSobelX_Or, "imgComboSatSobelX_Or", {"cmap":"gray"}), 
]

%matplotlib qt4
PlotImageRecords(pipelineImages)
#PlotImageRecords(pipelineImages)

In [9]:
imgOneCh = imgHSL[:,:,0]
min = 0
max = 255
imgBWMask = np.zeros_like(imgOneCh)
imgBWMask[(imgOneCh > 0) & (imgOneCh <= 255)] = 1


#%matplotlib qt4
#plt.figure(figsize=(12,8))
#plt.imshow(imgBWMask, cmap="gray")
#plt.show()


In [10]:
hls = cv2.cvtColor(imgIn, cv2.COLOR_RGB2HLS)
H = hls[:,:,0]
L = hls[:,:,1]
S = hls[:,:,2]
thresh = (0, 255)
binary = np.zeros_like(S)
binary[(S > thresh[0]) & (S <= thresh[1])] = 1

#%matplotlib qt4
#plt.figure(figsize=(12,8))
#plt.imshow(binary, cmap="gray")
#plt.show()
